In [1]:

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
from google.colab import drive

try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Drive mounting failed: {e}")

DATA_PATH = '/content/drive/MyDrive/FacialAgingProject/'
NUM_CLASSES = 4
IMG_SIZE = 224
RANDOM_STATE = 42

INITIAL_LR = 1e-3
FINE_TUNE_LR = 1e-5
BATCH_SIZE = 16
EPOCHS_STAGE1 = 40
EPOCHS_STAGE2 = 80

print("Loading dataset...")
try:
    X = np.load(DATA_PATH + 'X_data_224_rgb.npy')
    y_encoded = np.load(DATA_PATH + 'y_labels_one_hot.npy')
except FileNotFoundError:
    raise FileNotFoundError("Data files not found. Check your DATA_PATH or filenames.")

X = X.astype('float32') / 255.0
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=RANDOM_STATE, stratify=y_encoded
)
print(f"Dataset Loaded | Train: {X_train.shape}, Val: {X_val.shape}")

y_train_labels = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', classes=np.arange(NUM_CLASSES), y=y_train_labels)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

def build_model(input_shape=(224,224,3), num_classes=4):
    preprocess_input = tf.keras.applications.efficientnet.preprocess_input
    inputs = Input(shape=input_shape)
    x = preprocess_input(inputs)

    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.6)(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model, base_model

model, base_model = build_model()
model.summary()

print("\nStage 1: Training classification head ...")
model.compile(
    optimizer=Adam(learning_rate=INITIAL_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_stage1 = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ModelCheckpoint(DATA_PATH + 'stage1_best.h5', monitor='val_accuracy', save_best_only=True)
]

history_stage1 = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS_STAGE1,
    batch_size=BATCH_SIZE,
    class_weight=class_weights,
    callbacks=callbacks_stage1,
    verbose=2
)

print("\nStage 2: Fine-tuning EfficientNet layers ...")
model.load_weights(DATA_PATH + 'stage1_best.h5')
base_model.trainable = True  # Unfreeze all layers

optimizer = Adam(learning_rate=FINE_TUNE_LR)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

callbacks_stage2 = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint(DATA_PATH + 'final_efficientnet_model.h5', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-7)
]

history_stage2 = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS_STAGE2,
    batch_size=BATCH_SIZE,
    class_weight=class_weights,
    callbacks=callbacks_stage2,
    verbose=2
)

model.load_weights(DATA_PATH + 'final_efficientnet_model.h5')
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)

print("\nTraining Complete")
print(f"Final Validation Accuracy: {accuracy*100:.2f}%")
print(f"Final Validation Loss: {loss:.4f}")
print(f"Saved Model: {DATA_PATH}final_efficientnet_model.h5")


Mounted at /content/drive
Loading dataset...
Dataset Loaded | Train: (960, 224, 224, 3), Val: (240, 224, 224, 3)
Class Weights: {0: np.float64(1.0), 1: np.float64(1.0), 2: np.float64(1.0), 3: np.float64(1.0)}
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,378,535 (16.70 MB)

 Trainable params: 328,964 (1.25 MB)

...

60/60 - 7s - 118ms/step - accuracy: 0.8375 - loss: 0.4966 - val_accuracy: 0.8500 - val_loss: 0.4925 - learning_rate: 5.0000e-06
Epoch 27/80


60/60 - 5s - 91ms/step - accuracy: 0.8281 - loss: 0.5035 - val_accuracy: 0.8667 - val_loss: 0.4705 - learning_rate: 5.0000e-06
Epoch 28/80
60/60 - 5s - 78ms/step - accuracy: 0.8375 - loss: 0.5151 - val_accuracy: 0.8583 - val_loss: 0.4533 - learning_rate: 5.0000e-06
Epoch 29/80


60/60 - 7s - 110ms/step - accuracy: 0.8375 - loss: 0.5097 - val_accuracy: 0.8750 - val_loss: 0.4292 - learning_rate: 5.0000e-06
Epoch 30/80
60/60 - 5s - 78ms/step - accuracy: 0.8260 - loss: 0.5144 - val_accuracy: 0.8667 - val_loss: 0.4661 - learning_rate: 5.0000e-06
Epoch 31/80
60/60 - 5s - 75ms/step - accuracy: 0.8365 - loss: 0.4832 - val_accuracy: 0.8458 - val_loss: 0.4992 - learning_rate: 5.0000e-06
Epoch 32/80
60/60 - 5s - 80ms/step - accuracy: 0.8385 - loss: 0.4741 - val_accuracy: 0.8750 - val_loss: 0.4071 - learning_rate: 5.0000e-06
Epoch 33/80


60/60 - 7s - 121ms/step - accuracy: 0.8542 - loss: 0.4612 - val_accuracy: 0.8792 - val_loss: 0.4022 - learning_rate: 5.0000e-06
Epoch 34/80
60/60 - 4s - 74ms/step - accuracy: 0.8406 - loss: 0.4829 - val_accuracy: 0.8583 - val_loss: 0.4508 - learning_rate: 5.0000e-06
Epoch 35/80
60/60 - 5s - 76ms/step - accuracy: 0.8781 - loss: 0.4183 - val_accuracy: 0.8583 - val_loss: 0.4698 - learning_rate: 5.0000e-06
Epoch 36/80

Epoch 36: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
60/60 - 5s - 75ms/step - accuracy: 0.8594 - loss: 0.4423 - val_accuracy: 0.8667 - val_loss: 0.4388 - learning_rate: 5.0000e-06
Epoch 37/80
60/60 - 4s - 72ms/step - accuracy: 0.8427 - loss: 0.4507 - val_accuracy: 0.8625 - val_loss: 0.4249 - learning_rate: 2.5000e-06
Epoch 38/80
60/60 - 5s - 75ms/step - accuracy: 0.8615 - loss: 0.4596 - val_accuracy: 0.8792 - val_loss: 0.4134 - learning_rate: 2.5000e-06
Epoch 39/80
60/60 - 4s - 75ms/step - accuracy: 0.8573 - loss: 0.4379 - val_accuracy: 0.8792 - val_l

60/60 - 7s - 115ms/step - accuracy: 0.8760 - loss: 0.4045 - val_accuracy: 0.8917 - val_loss: 0.3794 - learning_rate: 1.2500e-06
Epoch 51/80

Epoch 51: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-07.
60/60 - 5s - 76ms/step - accuracy: 0.8698 - loss: 0.4002 - val_accuracy: 0.8917 - val_loss: 0.3757 - learning_rate: 1.2500e-06
Epoch 52/80
60/60 - 4s - 74ms/step - accuracy: 0.8656 - loss: 0.4229 - val_accuracy: 0.8875 - val_loss: 0.3789 - learning_rate: 6.2500e-07
Epoch 53/80
60/60 - 5s - 76ms/step - accuracy: 0.8729 - loss: 0.3784 - val_accuracy: 0.8750 - val_loss: 0.3734 - learning_rate: 6.2500e-07
Epoch 54/80
60/60 - 5s - 88ms/step - accuracy: 0.8771 - loss: 0.3969 - val_accuracy: 0.8750 - val_loss: 0.3687 - learning_rate: 6.2500e-07
Epoch 55/80
60/60 - 4s - 74ms/step - accuracy: 0.8562 - loss: 0.4153 - val_accuracy: 0.8792 - val_loss: 0.3657 - learning_rate: 6.2500e-07
Epoch 56/80
60/60 - 4s - 74ms/step - accuracy: 0.8708 - loss: 0.4164 - val_accuracy: 0.8833 - val_lo